In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

In [2]:
data  = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data.head()
Col = list(data.columns)
Col.remove('Personal Loan')
Col.append('Personal Loan')
data = data[Col].drop(columns='ZIP Code')

In [3]:
from sklearn.model_selection import train_test_split
X = data.values[:,:-1]
X = X.reshape(X.shape[0],1,X.shape[1])
Y = data.values[:,-1].reshape(X.shape[0],1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

In [4]:
w_inp = np.random.rand(1,12)
w_hid = np.random.rand(1,4)

In [5]:
def Sigmoid(x):
    s = 1 / (1+np.exp(-1*x))
    return s

In [6]:
def dri_Sigmoid(x):
    s = Sigmoid(x) * (1-Sigmoid(x))
    return s

In [7]:
def Predict(S,x_test,y_test):
    from sklearn.metrics import accuracy_score
    wI = S['w_inp']
    wH = S['w_hid']
    inp_out = (wI*x_test)
    inp_out = inp_out.reshape(inp_out.shape[0],4,3).sum(axis=2)
    inp_out = Sigmoid(inp_out)
    hid_out = wH*inp_out
    hid_out = hid_out.reshape(hid_out.shape[0],1,4).sum(axis=2)
    hid_out = np.floor(Sigmoid(hid_out))
    
    return accuracy_score(hid_out,y_test)*100

In [8]:
def D1toMat(sol):
    Z = {
        "w_inp" : (sol.reshape(4,4)[:3]).reshape(1,12),
        "w_hid" : sol.reshape(4,4)[-1]
    }
    return Z

In [9]:
def Fitness(sol,X,Y):
    fitness =[]
    for i in sol :
        Z = D1toMat(i)
        fitness.append(Predict(Z,X,Y))

    return fitness

In [10]:
rou = np.random.rand()

In [11]:
def Report(Weights,x_test,y_test):
    OUT = []
    for i in Weights:
        Z = D1toMat(i)
        fit = Fitness([i],x_test,y_test)[0]
        OUT.append((Z,fit))
    return OUT

In [12]:
#Population Creation
pop_size = 10
Pop =[]
for i in range(pop_size):
    temp = np.random.rand(1,16)
    Pop.append(temp)
fitness = np.array(Fitness(Pop,x_test,y_test)) * 0.01

Pop = np.array(Pop)
Column = sorted(list(set(list(Pop.reshape(160,)))))
Cost = np.zeros((160,160))
Pheromon = np.ones((160,160))


In [13]:
# Cost matrix Maker
for q in range(10):
    i = Pop[q] 
    for j in range(0,15):
        Cost[Column.index(i[0][j])][Column.index(i[0][j+1])] = 1 / fitness[q]

In [48]:
# Path Decider
def Cost_path(Cost,Phermon,start,ends,Column):
    warnings.filterwarnings('ignore')
    P =[]
    Sum = 0
    for q in ends:
        Start = Column.index(start)
        q1 = Column.index(q)
        Sum += (Phermon[Start][q1] * Cost[Start][q1] ) 
    Sum = Sum**(-1)
    for q in ends:
        val = (Phermon[Start][q1] * Cost[Start][q1]) * Sum
        P.append(val)
    temp  = np.random.rand()
    for r in range(len(P)):
        if P[r] >= temp:
            return ends[r]
    return ends[0]

In [49]:
# Neighbour Finder
def Neighbours(Pop,start):
    neigh = []
    for j in Pop:
        for i in range(16):
            temp = j[0][i]
            if(temp==start):
                if(i!=15 and j[0][i+1] not in neigh):
                    neigh.append(j[0][i+1])
                if(i!=0 and j[0][i-1] not in neigh):
                    neigh.append(j[0][i-1])
    return neigh

In [50]:
def Decay(rou,Cost,Pheromon):
    for i in range(Pheromon.shape[0]):
        for j in range(Pheromon.shape[1]):
            Pheromon[i][j] = (1-rou)*Pheromon[i][j] + Cost[i][j] 
    return Pheromon

In [51]:
def Pher_change(Pheromon,Start,Next,Column):
    Pheromon[Column.index(Start)][Column.index(Next)] +=1 
    return Pheromon

In [52]:
def func(x):
    return x[1]

In [53]:
def ACO(pop_size,Pop,Column,Cost,Pheromon,rou,x_test,y_test,trail=50):
    Final = []
    for i in tqdm(range(trail)):
        Start = np.random.randint(0,len(Column))
        temp_path =[]
        temp_path.append(Column[Start])
        Start = Column[Start]
        for _ in range(15):
            neigh = Neighbours(Pop,Start)
            Next = Cost_path(Cost,Pheromon,Start,neigh,Column)
            Pheromon  = Pher_change(Pheromon,Start,Next,Column)
            Pheromon = Decay(rou,Cost,Pheromon)
            temp_path.append(Next)
            Start = Next
        temp_path  = np.array(temp_path)
        # Final(Fitness([temp_path],x_test,y_test))
        Final.append(temp_path)
    Final  = list(np.array(Final).reshape(len(Final),1,16))
    return Report(Final,x_test,y_test)

In [54]:
Out = ACO(pop_size,Pop,Column,Cost,Pheromon,rou,x_test,y_test,1000)
Out_ACU = sorted(Out,key=func,reverse=True)[0]
print(Out_ACU)

100%|██████████| 1000/1000 [04:07<00:00,  4.04it/s]


({'w_inp': array([[0.47849354, 0.36812007, 0.61776524, 0.55873989, 0.16319922,
        0.51591551, 0.16319922, 0.51591551, 0.16319922, 0.51591551,
        0.16319922, 0.51591551]]), 'w_hid': array([0.16319922, 0.51591551, 0.16319922, 0.51591551])}, 88.8)


# END